In [2]:
from qtpg.team import Team
from qtpg.agent import Agent
from qtpg.trainer import Trainer
from qtpg.figure13 import Figure13
import numpy as np

In [3]:
# gp params
numLearners = 4
numAgents = 20
numRuns = 1
numGens = 100
numEpisodes = 50
gap = 0.3
# rl params
alpha, discount, epsilon = 0.9, 0.9, 0.1
# env params
memorySize = 20
legalMove = 0.1
illegalMove = -0.01
outOfBounds = -0.01
memoryRepeat = 0#-0.01
goalReached = 100

In [4]:
trainer = Trainer(gap, numLearners, numAgents, alpha, discount, epsilon)
# init env
env = Figure13(5, 5, (2, 4), (4, 0), memorySize, legalMove, illegalMove, outOfBounds, memoryRepeat, goalReached)
env.memoryRepeat = 0.0
# search space to make rules
trainer.generateRules(env)
print(trainer.rules)

[]


In [ ]:
for run in range(numRuns):
    # init agent population
    trainer = Trainer(gap, numLearners, numAgents, alpha, discount, epsilon)
    
    # init env
    env = Figure13(5, 5, (2, 4), (4, 0), memorySize, legalMove, illegalMove, outOfBounds, memoryRepeat, goalReached)
    env.memoryRepeat = 0.0
    # search space to make rules
    trainer.generateRules(env)
    
    trainer.createInitAgents()
    env.memoryRepeat = -0.01
    champs = []
    for gen in range(numGens):
        print(gen)
        env.reset()
        for agent in trainer.agents:
            print(agent.win)
            env.reset()
            episode_fitness = [] # new
            for ep in range(numEpisodes):
                if agent.win is not True:
                    env.reset()
                    fitness, win, states, seq = agent.evaluate_fitness(env)
                    episode_fitness.append(fitness)
#                     print(agent.team.q_table)
                    if win == True:
                        print('win stuff start')
                        print(states)
                        agent.win = True
                        for q_value in agent.team.q_table:
                            print(str(q_value['learner']) + ' ' + str(q_value['action']) + ' ' + str(q_value['q']))
                        for i in range(5):
                            env.reset()
                            agent.team.victory_lap(env, seq)
                        print('\n\n\n')
                        for q_value in agent.team.q_table:
                            print(str(q_value['learner']) + ' ' + str(q_value['action']) + ' ' + str(q_value['q']))
                        env.reset()
#                         agent.team.epsilon = 0.0
                        score, win, states = agent.replay(env)
                        print(agent.team.q_table)
                        print('win stuff end!')
                        print('REPLAY')
                        print(score)
                        print(win)
                        if not win:
                            agent.win = False
                        else:
                            champs.append(agent)
                            print('win replay!')
                        
                        print(states)
                        print('!!!!')
            # average fitness over episodes...
            average_fitness = 0
            average_fitness = np.mean(episode_fitness)
            print(average_fitness)
            agent.fitness = average_fitness
        trainer.evolve()

In [ ]:
print(len(champs))
champ_index = 0
champs[champ_index].team.epsilon = 0.0

for q_value in champs[champ_index].team.q_table:
    print(str(q_value['learner']) + ' ' + str(q_value['action']) + ' ' + str(q_value['q']))

wins = 0
for i in range(100):
    env.reset()
    score, win, states = champs[champ_index].replay(env)
    print(win)
    if win:
        print(states)
        wins += 1
    print(score)
print(wins)